In [1]:
import json
import requests
import os
from datetime import datetime
from dateutil import tz
import boto3

CLT = tz.gettz("America/Santiago")

URL = "https://www.ine.gob.cl/estadisticas/sociales/mercado-laboral/ocupacion-y-desocupacion/getArchivos/"
#RAW_BUCKET = os.getenv("RAW_BUCKET")


In [6]:

now = datetime.now(tz=CLT)
year = now.year
month = str(now.month).zfill(2)

download_filter = []
#event.get("download_filters", [])

FOLDER_ID = '1270d9b3-5bcb-424c-b08a-180fb99dc5ab'
#event.get("folder_id")

request_body = {
    "idFolder" : FOLDER_ID,
    "Content-Type" : "multipart/form-data",
    "Accept" : "*/*"
}
folderInfo = requests.post(url=URL, data=request_body, timeout=60)

if folderInfo.status_code != 200:
    raise Exception("No se pudo obtener la información del folder del INE")

children = folderInfo.json()["documento"]

data = []
for child in children:
    print("Convirtiendo", child)
    item = {
        "title" : child["Titulo"].replace(":", "-"),
        "url" : child["Url"].replace("http", "https"),
        "tipo" : child["Tipo"]
    }
    if len(download_filter) == 0 or child["Titulo"] in download_filter:
        print("Agregado archivo", child["Titulo"], "a la lista de descargas")
        data.append(item)

s3 = boto3.resource("s3")
#Descargar datos
for archivo in data:
    print("descargando", archivo["url"])
    ine_file = requests.get(archivo["url"], timeout=60)

    file_name = archivo["title"] + archivo["tipo"].lower()
    
    file_name = file_name.strip()

    #open("ine/", "wb").write(ine_file.content)

    #s3.Object(RAW_BUCKET, f"landing/INE/year={year}/month={month}/{ file_name }").upload_file("/tmp/" + file_name)


Convirtiendo {'Titulo': 'IR-ICMO Nominal General (Indices y Variaciones)', 'Descripcion': '', 'Peso': '39936', 'Fecha': None, 'Url': 'http://www.ine.gob.cl/docs/default-source/sueldos-y-salarios/cuadros-estadisticos/iricmo-base-anual-2016/series-mensuales/ir-icmo-nominal-general.xls?sfvrsn=3dd3a011_86', 'Tipo': '.XLS', 'Periodo': None}
Agregado archivo IR-ICMO Nominal General (Indices y Variaciones) a la lista de descargas
Convirtiendo {'Titulo': 'IR Real General (Índices y Variaciones, según IPC Base 2018=100)', 'Descripcion': '', 'Peso': '39424', 'Fecha': None, 'Url': 'http://www.ine.gob.cl/docs/default-source/sueldos-y-salarios/cuadros-estadisticos/iricmo-base-anual-2016/series-mensuales/ir-real-general-(índices-y-variaciones-según-ipc-base-2018=100).xls?sfvrsn=1bd25df_90', 'Tipo': '.XLS', 'Periodo': None}
Agregado archivo IR Real General (Índices y Variaciones, según IPC Base 2018=100) a la lista de descargas
Convirtiendo {'Titulo': 'IR Nominal por Actividad Económica (Indices y Va

OSError: [Errno 22] Invalid argument: 'ine/'